In [1]:
from fv3fit.reservoir import (
    ReservoirHyperparameters,
    Reservoir, 
    ReservoirComputingReadout, 
    HybridReservoirComputingModel,
    RankDivider,
)

from fv3fit.reservoir.transformers import DoNothingAutoencoder
from fv3fit.reservoir.readout import combine_readouts

In [2]:
autoencoder = DoNothingAutoencoder([79])

In [3]:

hyperparameters = ReservoirHyperparameters(
    state_size=3000,
    adjacency_matrix_sparsity=0.9,
    spectral_radius=0.5,
    input_coupling_sparsity=0,
)

rank_divider = RankDivider([8, 8], ["x", "y"], [52, 52, 79], overlap=2)


In [4]:
rank_divider.subdomain_size_with_overlap

7900

In [5]:
input_size_subdomain = rank_divider.subdomain_size_with_overlap * rank_divider.n_subdomains
output_size_subdomain = rank_divider.subdomain_xy_size_without_overlap ** 2 * 79 * rank_divider.n_subdomains

In [6]:
input_size_subdomain * output_size_subdomain * 8 / 1024**3

685.65673828125

685 GB for single C48 model tile.... with full readout matrix resolution. Need to stack matrices in sparse matrix to get a true readout layer.  But! We have a combine_readouts that loads the block diagonal matrix into

In [9]:
rank_divider.get_subdomain_extent(True)

(10, 10, 79)

readout takes flat vector of subdomains stacked column by column

input @ coefficenits

7900 <> flattened state x 1 ?? @ 

7900 x ns
hidden_state x ns

(7900 + hidden_state) x ns
nf

nf*ns 1D array

What are the readout weights?

Ax = y

nf*ns x nf*ns

training inputs X is ntimes x nf*ns


Ridge regression

We have a matrix of input X and a matrix of output Y.  We want to use regression to find a matrix W such that Y = XW.  This is a linear regression problem.  Ridge regression is a regularized version of linear regression.  The regularization term is a penalty on the size of the coefficients.  The ridge coefficients minimize a penalized residual sum of squares,

If X is nsamples by nfeatures, then W is nfeatures by noutputs

W = (X^T X + alpha*I)^-1 X^T Y

X^T X is nfeatures x nfeatures

W is nfeatures x noutputs

XW is nsamples x noutputs





In [ ]:


    reservoir = Reservoir(hyperparameters, input_size=input_size)

    # multiplied by the number of subdomains since it's a combined readout
    readout = ReservoirComputingReadout(
        coefficients=np.random.rand(
            state_size * 4 + hybrid_input_size_per_subdomain * 4, output_size * 4
        ),
        intercepts=np.random.rand(output_size * 4),
    )

    hybrid_predictor = HybridReservoirComputingModel(
        input_variables=["a", "b"],
        output_variables=["a", "b"],
        hybrid_variables=["a", "b"],
        reservoir=reservoir,
        readout=readout,
        rank_divider=divider,
        autoencoder=autoencoder,
    )
    hybrid_predictor.reset_state()

    return hybrid_predictor